In [8]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

countries = spark.read.csv("/workspaces/PySpark/datatrainadls2/raw/countries.csv", header=True)

In [2]:
countries.show()

+----------+-------------------+--------------------+------------+----------+----------------+----------+--------+---------+-------------+----------------------+----------------------+
|COUNTRY_ID|               NAME|         NATIONALITY|COUNTRY_CODE|ISO_ALPHA2|         CAPITAL|POPULATION|AREA_KM2|REGION_ID|SUB_REGION_ID|INTERMEDIATE_REGION_ID|ORGANIZATION_REGION_ID|
+----------+-------------------+--------------------+------------+----------+----------------+----------+--------+---------+-------------+----------------------+----------------------+
|         1|        Afghanistan|              Afghan|         AFG|        AF|           Kabul|  38041754|  652230|       30|           30|                  NULL|                    30|
|         2|            Albania|            Albanian|         ALB|        AL|          Tirana|   2880917|   28748|       20|           70|                  NULL|                    20|
|         3|            Algeria|            Algerian|         DZA|        D

In [6]:
spark.conf.set("fs.azure.account.key.datatrainadls2.dfs.core.windows.net", "2ivvCahbvGLMNawvnUOfI2i5a9mJdPQLR1tNGbAEkNbRS6t17ZZXL1U09QPK0cBy4+/AYi1OtOOh+AStBegyAg==")

In [7]:
# create temp view
countries.createOrReplaceTempView("countries")

# write to parquet with option overwrite to overwrite existing file
# spark.sql("SELECT * FROM countries").write.mode("overwrite").csv("/workspaces/PySpark/datatrainadls2/bronze/countries_new.csv")
# countries.write.mode("overwrite").parquet("/workspaces/PySpark/input/bronze/ountries_parquet")
# write to delta
countries.write.mode("overwrite").format("delta").save("/workspaces/PySpark/datatrainadls2/bronze/countries_dt")